In [13]:
from lnn import Model

model = Model()

In [14]:
from lnn import Predicate

grecian = Predicate(name="grecian")
mortal = Predicate(name="mortal")

model.add_formulae(grecian, mortal)

In [15]:
from lnn import Variable, Implies

x = Variable("x")

grecians_are_mortals = Implies(grecian(x), mortal(x), name="grecians_are_mortals")

model.add_formulae(grecians_are_mortals)

In [16]:
from lnn import Fact

model.add_facts(
    {
        "grecian": {
            "Zeus": Fact.TRUE,
            "Socrates": Fact.TRUE,
            "Confucius": Fact.FALSE,
            "Great Pyramid of Giza": Fact.FALSE,
        },
        "mortal": {
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
            "Confucius": Fact.TRUE,
            "Great Pyramid of Giza": Fact.FALSE,
        },
    }
)


In [17]:
from lnn import Direction

steps, facts_inferred = model.infer(direction=Direction.UPWARD)

In [18]:
model.print()


***************************************************************************
                                LNN Model

OPEN   Implies: grecians_are_mortals(x) 
'Socrates'                                                  TRUE (1.0, 1.0)
'Zeus'                                                     FALSE (0.0, 0.0)
'Confucius'                                                 TRUE (1.0, 1.0)
'Great Pyramid of Giza'                                     TRUE (1.0, 1.0)

OPEN   Predicate: mortal(x0) 
'Socrates'                                                  TRUE (1.0, 1.0)
'Zeus'                                                     FALSE (0.0, 0.0)
'Confucius'                                                 TRUE (1.0, 1.0)
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)

OPEN   Predicate: grecian(x0) 
'Socrates'                                                  TRUE (1.0, 1.0)
'Zeus'                                                      TRUE (1.0, 1.0)
'Confucius'     